In [1]:
import torch
# 학습된 모델 load. nvidia처럼 모델 동작에 필요한 것들 import. checkpoint load.
# github repository의 코드를 가져올 수 있게 wrapping 해줌. 
model = torch.hub.load('pytorch/vision:v0.9.0', 'deeplabv3_resnet101', pretrained=True)
# model.eval()

model.last_linear = torch.nn.Sequential(
    torch.nn.BatchNorm1d(2048),
    torch.nn.Dropout(p=0.25),
    torch.nn.Linear(in_features=2048, out_features=2048),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1),
    torch.nn.Dropout(p=0.5),
    torch.nn.Linear(in_features=2048, out_features=1103),
)

Using cache found in /home/ag/.cache/torch/hub/pytorch_vision_v0.9.0


In [2]:
import torchvision.transforms.functional as TF

def train_model(model, 
                data_loader, 
                dataset_size, 
                optimizer, 
                scheduler, 
                num_epochs):
    
    criterion = nn.BCEWithLogitsLoss()
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        scheduler.step()
        model.train()

        running_loss = 0.0
        # Iterate over data.
        for origin, label in data_loader:
            inputs = TF.to_tensor(origin)
            labels = TF.to_tensor(label)
            inputs = inputs.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / dataset_size
        print('Loss: {:.4f}'.format(epoch_loss))
    return model

In [5]:
import os
from PIL import Image

origin_path = './data/Original_data/'
label_path = './data/Labeled_data/'

if os.listdir(origin_path) == os.listdir(label_path):
    file_list = os.listdir(origin_path)
else:
    print('error')


import torchvision.transforms.functional as TF
train_dataset_loader = [((Image.open(origin_path + file).convert('RGB')), 
                         Image.open(label_path + file).convert('RGB')) 
              for file in file_list]

In [6]:
import torch
from torchvision import transforms

# define some re-usable stuff
IMAGE_SIZE = 224
NUM_CLASSES = 1103
BATCH_SIZE = 32
device = torch.device("cuda:0") # cuda 장치 객체로 gpu 상에 tensor 생성. 
# IMG_MEAN = model_ft.mean
# IMG_STD = model_ft.std

# path = './data/Original_data/'

# train_dataset_loader = torch.utils.data.DataLoader(path, batch_size = 2, shuffle = True)


# push model to device
model_ft = model.to(device)

In [7]:
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler

# 파인튜닝할 레이어 정의. 
plist = [
        #{'params': model_ft.layer4.parameters(), 'lr': 1e-5},
        {'params': model_ft.last_linear.parameters(), 'lr': 5e-3}
        ]
optimizer_ft = optim.Adam(plist, lr=0.001)
lr_sch = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)


model_ft = train_model(model_ft,
                       train_dataset_loader,
                       len(train_dataset_loader),
                       optimizer_ft,
                       lr_sch,
                       num_epochs=20)

torch.save(model_ft.state_dict(), "model.bin")

Epoch 0/19
----------


/home/ag/venv_capstone/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 3, 7, 7], but got 3-dimensional input of size [3, 720, 960] instead